# Gaplock Plot Explorer
This notebook loads the CSV/JSON artifacts we dump under `plots/<scenario>/<run>/` and renders attacker/defender paths or heat maps.
1. Run training to produce `paths_<run>.csv` + `run_config_<run>.json`.
2. Point `run_dir` below to the folder that contains those files.
3. Execute the plotting cells to visualize trajectories.

In [1]:
from pathlib import Path
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set this to the plots/<scenario>/<run>/<timestamp> directory
run_dir = Path('plots') / 'gaplock_td3'
while run_dir.exists() and run_dir.is_dir():
    csv_candidates = sorted(run_dir.glob('paths_*.csv'))
    json_candidates = sorted(run_dir.glob('run_config_*.json'))
    break
if not csv_candidates or not json_candidates:
    raise FileNotFoundError('No paths/run_config files found under ' + str(run_dir))
paths_csv = csv_candidates[-1]
config_json = json_candidates[-1]
print('Using', paths_csv)
print('Config', config_json)
df = pd.read_csv(paths_csv)
with open(config_json) as fh:
    run_cfg = json.load(fh)

NameError: name 'csv_candidates' is not defined

In [ ]:
episode = int(df['episode'].min())
agent = 'car_0'
sel = df[(df['episode'] == episode) & (df['agent_id'] == agent)]
plt.figure(figsize=(6,6))
plt.plot(sel['x'], sel['y'], label=f'{agent} ep {episode}')
plt.axis('equal')
plt.legend()
plt.title('Trajectory')
plt.show()

In [ ]:
agent = 'car_0'
agent_df = df[df['agent_id'] == agent]
plt.figure(figsize=(6,6))
sns.kdeplot(data=agent_df, x='x', y='y', fill=True, cmap='magma', thresh=0.05)
plt.axis('equal')
plt.title(f'Heat map for {agent}')
plt.show()